# Merge

Used when dataframes to be joined are not in exact same order or format. i.e., we cannot simply concatenate them

(analogous to JOIN command in sql)

**merge() method takes in a pivotal argument 'how' which takes three values:**
- **Inner**
- **Outer**
- **Left or Right**

**The main idea behind this argument 'how' is to decide how to deal with information only present in one of the joined tables**

In [40]:
# PARAMETERS

# on : on what column to merge dataframes (on column should be unique per row)
# how: inner/outer/left/right
# how is inner by default

In [1]:
import pandas as pd
import numpy as np

In [7]:
data_one = {'A':['A0','A1','A2','A3'],'B':['B0','B1','B2','B3']}
data_two = {'C':['C0','C1','C2','C3'],'D':['D0','D1','D2','D3']}
one = pd.DataFrame(data_one)
one

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [8]:
two = pd.DataFrame(data_two)
two

,C,D
0,C0,D0
1,C1,D1
2,C2,D2
3,C3,D3


In [11]:
two.columns = one.columns
df = pd.concat([one,two])
df.index = range(1,9)
df

,A,B
1,A0,B0
2,A1,B1
3,A2,B2
4,A3,B3
5,C0,D0
6,C1,D1
7,C2,D2
8,C3,D3


In [36]:
registrations = pd.DataFrame({'reg_id':[1,2,3,4],'name':['Andrew','Bobo','Claire','David']})
logins = pd.DataFrame({'log_id':[1,2,3,4],'name':['Xavier','Andrew','Yolanda','Bobo']})

In [14]:
registrations

,reg_id,name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [16]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


## Inner merge
**how: inner : result will be set of records that match in both tables (intersection)**

In [19]:
pd.merge(registrations,logins,how = 'inner',on = 'name')

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


In [29]:
# order in which dataframes are passed as arguments does not impact result 
# but it impacts the sorting (row wise of columns wise)
pd.merge(logins,registrations,how = 'inner',on = 'name')

,log_id,name,reg_id
0,2,Andrew,1
1,4,Bobo,2


## Left merge

Match up AND include all rows from Left Table. Show everyone who registered on Left Table, if they don't have info in the right table, then fill with NaN.

In a venn diagram with sets Left and Right, how = left will print the data in Left set

In [24]:
# order in which dataframes are passed as arguments is important

pd.merge(registrations,logins,how = 'left',on = 'name')

,reg_id,name,log_id
0,1,Andrew,2.0
1,2,Bobo,4.0
2,3,Claire,NaN
3,4,David,NaN


## Right merge

Match up AND include all rows from Right Table. Show everyone who registered on Right Table, if they don't have info in the left table, then fill with NaN.

In a venn diagram with sets Left and Right, how = right will print the data in Right set

In [23]:
pd.merge(registrations,logins,how = 'right',on = 'name')

,reg_id,name,log_id
0,NaN,Xavier,1
1,1.0,Andrew,2
2,NaN,Yolanda,3
3,2.0,Bobo,4


In [25]:
# on argument is not necessary to mention if the choice is obvious
# i.e., only one unique columns is shared between the dataframes

pd.merge(left = registrations,right = logins,how = 'left')

,reg_id,name,log_id
0,1,Andrew,2.0
1,2,Bobo,4.0
2,3,Claire,NaN
3,4,David,NaN


## Outer merge
how: outer : result will be set of records that is present in both tables
i.e., union of sets

In [26]:
pd.merge(registrations,logins,how = 'outer')

,reg_id,name,log_id
0,1.0,Andrew,2.0
1,2.0,Bobo,4.0
2,3.0,Claire,NaN
3,4.0,David,NaN
4,NaN,Xavier,1.0
5,NaN,Yolanda,3.0


In [30]:
# order in which dataframes are passed as arguments does not impact result 
# but it impacts the sorting (row wise of columns wise)
pd.merge(logins,registrations,how='outer')

,log_id,name,reg_id
0,1.0,Xavier,NaN
1,2.0,Andrew,1.0
2,3.0,Yolanda,NaN
3,4.0,Bobo,2.0
4,NaN,Claire,3.0
5,NaN,David,4.0


## Merging using index

In [31]:
# PARAMETERS

# left_on : left dataframe column to be looked at for merging
# right_on : right dataframe column to be looked at for merging
# left_index : use left dataframe index for merging (True or False)
# right_index : use right dataframe index for merging (True or False)

In [43]:
registrations_new = registrations.set_index('name')
registrations_new

,reg_id
name,
Andrew,1
Bobo,2
Claire,3
David,4


In [44]:
registrations

,reg_id,name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [46]:
logins_new = logins.set_index('name')
logins_new

,log_id
name,
Xavier,1
Andrew,2
Yolanda,3
Bobo,4


In [49]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


- **FOLLOWING EXAMPLES ILLUSTRATE DIFFERENT WAYS TO GET INNER MERGE OUTPUT**

In [41]:
pd.merge(registrations_new,logins,how='inner',
         right_on = 'name',left_index=True)

,reg_id,log_id,name
1,1,2,Andrew
3,2,4,Bobo


In [42]:
pd.merge(logins,registrations_new,how='inner',
        left_on='name',right_index=True)

,log_id,name,reg_id
1,2,Andrew,1
3,4,Bobo,2


In [48]:
pd.merge(logins_new,registrations_new,left_index=True,right_index=True)

,log_id,reg_id
name,,
Andrew,2,1
Bobo,4,2


In [53]:
# name of the unique columns is not same

registrations.columns = ['reg_id','reg_name']
registrations

,reg_id,reg_name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [55]:
# merging when name of the unique column is not same

result = pd.merge(registrations,logins,
         left_on = 'reg_name' , right_on = 'name')

result

,reg_id,reg_name,log_id,name
0,1,Andrew,2,Andrew
1,2,Bobo,4,Bobo


In [58]:
# cleaning above result table

#STEP 1 : dropping the last columns
result.drop('name',axis=1,inplace=True)

In [62]:
#STEP 2 : checking names of existing columns
result.columns

Index(['reg_id', 'reg_name', 'log_id'], dtype='object')

In [64]:
#STEP 3 : updating 'reg_name' to 'name'
result.columns = ['reg_id', 'name', 'log_id']

In [65]:
#STEP 4 : checking output
result

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


- **SITUATION WHERE COLUMN NAMES ARE SAME AFTER MERGING**

In [67]:
registrations.columns

Index(['reg_id', 'reg_name'], dtype='object')

In [68]:
registrations.columns = ['id', 'name']
registrations

,id,name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [69]:
logins.columns

Index(['log_id', 'name'], dtype='object')

In [71]:
logins.columns = ['id', 'name']
logins

,id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [72]:
pd.merge(registrations,logins,
        how = 'inner', on = 'name')

,id_x,name,id_y
0,1,Andrew,2
1,2,Bobo,4


In [75]:
# adding suffixes parameter to replace x and y

pd.merge(registrations,logins,
        how = 'inner', on = 'name',
        suffixes = ('_reg','_log'))

,id_reg,name,id_log
0,1,Andrew,2
1,2,Bobo,4
